In [ ]:
import my_module as mm
netsA = []
netsB = []
convs = [8, 16, 32, 64]
fcs = [152, 38, 14]
device = mm.torch.device('cuda' if mm.torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
count = 0

DIR_PATH_PADDING = './model/padding/'
DIR_PATH_STRIDE = './model/stride/'

for padding in range(3, 7):
    net = mm.Net(1, 350, 350, 7, convs, fcs, padding = padding)
    PATH = DIR_PATH_PADDING + "padding_" + str(padding)
    net.load_state_dict(mm.torch.load(PATH))
    netsA.append(net)

for stride in range(1, 6):
    net = mm.Net(1, 350, 350, 7, convs, fcs, stride = stride)
    PATH = DIR_PATH_PADDING + "padding_" + str(padding)
    net.load_state_dict(mm.torch.load(PATH))
    netsB.append(net)

print("Done")

In [ ]:
categories = [
    "anger",
    "surprise",
    "disgust",
    "fear",
    "neutral",
    "happiness",
    "sadness",
    "contempt"
]

X, y = mm.read_pic(mm.pd.read_csv('data/legend.csv'), categories)
seed = 1234
Xtr, Xva, ytr, yva = mm.train_test_split(X, y, test_size=0.2, random_state=seed)
trainloader, testloader = mm.get_loader(Xtr, Xva, ytr, yva)
print("Done")

In [ ]:
training_error_A = []
testing_error_A = []
count_A = []
count = 2
for net in netsA:
    if mm.torch.cuda.is_available():
        net.cuda()
    training_error_A.append(1 - mm.accuracy(device, net, trainloader))
    testing_error_A.append(1 - mm.accuracy(device, net, testloader))
    count += 1
    count_A.append(count)
    print("Done for stride", count)

In [ ]:
training_error_B = []
testing_error_B = []
count_B = []
count = 0
for net in netsB:
    if mm.torch.cuda.is_available():
        net.cuda()
    training_error_B.append(1 - mm.accuracy(device, net, trainloader))
    testing_error_B.append(1 - mm.accuracy(device, net, testloader))
    count += 1
    count_B.append(count)
    print("Done for padding", count)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1,2, figsize=(8,4))

ax[0].plot(count_A, training_error_A, label="Training error")
ax[0].plot(count_A, testing_error_A, label="Testing error")
ax[0].legend()
ax[1].plot(count_B, training_error_B, label="Training error")
ax[1].plot(count_B, testing_error_B, label="Testing error")
ax[1].legend()